In [21]:
import pandas as pd
import xml.etree.ElementTree as ET
from datetime import datetime

#make into function later
station_id = "080250"
path = "VDOT Capstone Files/"
volume_file = path + f"Station - {station_id} - Volume - 2021-2024.csv"
speed_file = path + f"Station - {station_id} - Speed - 2021-2024.csv"
output_file = f"output_flows/flows_{station_id}.rou.xml"

volume_df = pd.read_csv(volume_file, parse_dates=["STARTDATE1"])
volume_df.head()

,LINKID,STARTDATE1,DIRECTION,COUNTERNUMBER,LANE,VOL,AXLES,QUAL
0,80250,2021-01-01 00:00:00,N,1,1,15,NaN,4
1,80250,2021-01-01 00:00:00,N,1,2,30,NaN,4
2,80250,2021-01-01 00:00:00,N,1,3,10,NaN,4
3,80250,2021-01-01 00:15:00,N,1,1,18,NaN,4
4,80250,2021-01-01 00:15:00,N,1,2,20,NaN,4


In [6]:
# speed files have all the info that volume does, so only speed ones are necessary
speed_df = pd.read_csv(speed_file, parse_dates=["STARTDATE1"])
speed_df.head()

,LINKID,STARTDATE1,DIRECTION,COUNTERNUMBER,LANE,MPH_0_15,MPH_15_25,MPH_25_30,MPH_30_35,MPH_35_40,...,MPH_50_55,MPH_55_60,MPH_60_65,MPH_65_70,MPH_70_75,MPH_75_80,MPH_80_85,MPH_85_UP,VOL1,QUAL
0,80250,2021-01-01 00:00:00,N,1,1,0,0,0,0,0,...,3,2,4,3,0,0,0,0,15,4
1,80250,2021-01-01 00:00:00,N,1,2,0,0,0,0,0,...,0,0,3,10,5,8,4,0,30,4
2,80250,2021-01-01 00:00:00,N,1,3,0,0,0,0,0,...,0,0,1,3,3,2,1,0,10,4
3,80250,2021-01-01 00:15:00,N,1,1,0,0,0,0,0,...,1,6,7,2,1,0,0,0,18,4
4,80250,2021-01-01 00:15:00,N,1,2,0,0,0,0,0,...,0,0,1,7,4,6,2,0,20,4


In [12]:
speed_df.columns

Index(['LINKID', 'STARTDATE1', 'DIRECTION', 'COUNTERNUMBER', 'LANE',
       'MPH_0_15', 'MPH_15_25', 'MPH_25_30', 'MPH_30_35', 'MPH_35_40',
       'MPH_40_45', 'MPH_45_50', 'MPH_50_55', 'MPH_55_60', 'MPH_60_65',
       'MPH_65_70', 'MPH_70_75', 'MPH_75_80', 'MPH_80_85', 'MPH_85_UP', 'VOL1',
       'QUAL'],
      dtype='object')

In [20]:
speed_bins = {
    "MPH_0_15": 7.5, "MPH_15_25": 20, "MPH_25_30": 27.5, "MPH_30_35": 32.5, "MPH_35_40": 37.5, "MPH_40_45": 42.5, 
    "MPH_45_50": 47.5, "MPH_50_55": 52.5, "MPH_55_60": 57.5, "MPH_60_65": 62.5, "MPH_65_70": 67.5, "MPH_70_75": 72.5,
    "MPH_75_80": 77.5, "MPH_80_85": 82.5, "MPH_85_UP": 87.5
}

start_time = speed_df["STARTDATE1"].min() # for comparisons
# https://sumo.dlr.de/docs/Definition_of_Vehicles%2C_Vehicle_Types%2C_and_Routes.html#repeated_vehicles_flows
flows = []
for _, row in speed_df.iterrows():
    timestamp = row["STARTDATE1"]
    lane = int(row["LANE"])
    vol = row["VOL1"]
    begin = int((timestamp - start_time).total_seconds())
    end = begin + 900  # 15 min interval
    vehsPerHour = vol * 4

    total_cars = 0
    weighted_speed = 0
    for col, mph in speed_bins.items():
        count = row[col]
        total_cars += count
        weighted_speed += count * mph

    if total_cars == 0:
        continue # avoid divide by 0 or putting in an empty flow
    avg_speed = weighted_speed / total_cars

    flow_id = f"{station_id}_{timestamp.strftime('%Y%m%d_%H%M')}_LANE{lane}"
    flow = f'<flow id="{flow_id}" begin="{begin}" end="{end}" from="edge_in" to="edge_out" vehsPerHour="{round(vehsPerHour,2)}" speed="{round(avg_speed,2)}" departLane="free"> </flow>'
    flows.append(flow)

print(flows[0])

<flow id="080250_20210101_0000_LANE1" begin="0" end="900" from="edge_in" to="edge_out" vehsPerHour="60" speed="57.83" departLane="free"> </flow>


In [22]:
with open(output_file, "w") as f:
    f.write('<routes>\n')
    for flow in flows:
        f.write(f"  {flow}\n")
    f.write('</routes>\n')